# Environments

En JOS a los <b>procesos</b> se les llama <b>environments</b>. Asi que cuando hagamos referencia a un <b>environment</b> estaremos hablando de un <b>proceso</b> y viceversa.

### Estructura de un Environment 

La siguiente es la estructura de un proceso. Como podemos ver tiene diferentes cosas como el <b>env_id</b>, <b>env_parent_id</b> osea el <b>id</b> el proceso y el de su padre. Ademas tiene una estructura para guardarse los registros que utiliza, esto es <b>env_tf</b>. Luego tambien tiene una variable que guarda su estado, esta es <b>env_status</b>. Luego tenemos <b>env_runs</b> que guarda la cantidad de veces que este proceso fue corrido. Y por ultimo <b>env_pgdir</b> que es el directorio de paginas que usa este proceso.

```c
struct Env {
	struct Trapframe env_tf;	// Saved registers
	struct Env *env_link;		// Next free Env
	envid_t env_id;			// Unique environment identifier
	envid_t env_parent_id;		// env_id of this env's parent
	enum EnvType env_type;		// Indicates special system environments
	unsigned env_status;		// Status of the environment
	uint32_t env_runs;		// Number of times environment has run

	// Address space
	pde_t *env_pgdir;		// Kernel virtual address of page dir
};
```

#### Estados posibles para un Environment

```c
enum {
	ENV_FREE = 0,
	ENV_DYING,
	ENV_RUNNABLE,
	ENV_RUNNING,
	ENV_NOT_RUNNABLE
};
```